In this notebook, we are going to load the data we have prepared in the previous notebook in our Data Warehouse based on Postgres. We will also create a function to load the data in the future.

In [77]:
import numpy as np
import pandas as pd
import psycopg2

In [61]:
def insert_row(table, row):
    """ insert a new row into the table """
    sql = "INSERT INTO " + table + " VALUES (%s" + ", %s" * (len(row)-1) + ");"
    conn = None
    vendor_id = None
    try:
        # connect to the PostgreSQL database
        conn = psycopg2.connect(database="moroccan_banks", user="postgres" , password="postgres" , host="localhost")
        # create a new cursor
        cur = conn.cursor()
        # execute the INSERT statement
        cur.execute(sql, row)
        # commit the changes to the database
        conn.commit()
        # close communication with the database
        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()

In [62]:
def insert_rows(table, rows):
    """ insert multiple rows into the table  """
    sql = "INSERT INTO " + table + " VALUES (%s" + ", %s" * (len(rows[0])-1) + ");"
    conn = None
    try:
        # connect to the PostgreSQL database
        conn = psycopg2.connect(database="moroccan_banks", user="postgres" , password="postgres" , host="localhost")
        # create a new cursor
        cur = conn.cursor()
        # execute the INSERT statement
        cur.executemany(sql, rows)
        # commit the changes to the database
        conn.commit()
        # close communication with the database
        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()

In [63]:
# test the function for 1 row
# insert_row("fact_table" ,(1, 1, '2021-04-29', 4.5, 7, 3.4, 32, 18))

# test the function for multiple rows
# insert_rows('reviews', [(2, 'test', 'test', 'test'), (3, 'test', 'test', 'test')])

In [79]:
# load our datasets
banks = pd.read_excel('../data/processed/banks.xlsx')
reviews = pd.read_excel('../data/processed/reviews_cleaned.xlsx')

In [80]:
banks.columns, reviews.columns

(Index(['bank_id', 'bank_title', 'bank_category', 'bank_adress', 'bank_city',
        'bank_phone', 'bank_rank', 'bank_reviews_count', 'bank_score',
        'bank_latitude', 'bank_longitude'],
       dtype='object'),
 Index(['review_id', 'bank_id', 'review_date', 'review_stars', 'review_text',
        'reviewer_name', 'review_sentiment'],
       dtype='object'))

In [81]:
# get data for each table
banks_table = banks[['bank_id', 'bank_title', 'bank_category', 'bank_adress', 'bank_city', 'bank_phone', 'bank_rank']]

In [82]:
reviews_table = reviews[['review_id', 'review_text', 'reviewer_name', 'review_sentiment']]

In [86]:
facts_table = pd.merge(reviews, banks, on='bank_id', how='inner')[['review_id', 'bank_id', 'review_date', 'review_stars', 'bank_reviews_count', 'bank_score','bank_latitude', 'bank_longitude']]

In [84]:
banks_table.columns, reviews_table.columns, fact_table.columns

(Index(['bank_id', 'bank_title', 'bank_category', 'bank_adress', 'bank_city',
        'bank_phone', 'bank_rank'],
       dtype='object'),
 Index(['review_id', 'review_text', 'reviewer_name', 'review_sentiment'], dtype='object'),
 Index(['review_id', 'bank_id', 'review_date', 'review_stars',
        'bank_reviews_count', 'bank_score', 'bank_latitude', 'bank_longitude'],
       dtype='object'))

Our dataset seem to be matching the schema we have created for our data warehouse. It is time to load the data.

In [91]:
# load data into the database
insert_rows('banks', banks_table.values.tolist())
insert_rows('reviews', reviews_table.values.tolist())
insert_rows('facts', facts_table.values.tolist())